In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/rt-titles - keywords.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df

,Number,Title of the paper,Abstract,Keyword 1,Keyword 2,Keyword 3,Keyword 4,Keyword 5,Keyword Eric 1,Keyword Eric 2,Keyword Eric 3,Keyword Eric 4,Keyword Eric 5,Unnamed: 13
0,Article 1,Evaluative feedback isn’t enough: harnessing t...,Most research on feedback in higher education ...,Consequential feedback mechanisms,Professional/disciplinary simulations,Audience-responsive assessment,Ecological feedback literacy,Human-nonhuman actor interactions,NaN,College Instruction,Student Evaluation,Educational Change,Program Improvement,Faculty Development
1,Article 2,The paradox of inclusive assessment,Higher education institutions are increasingly...,Inclusive assessment paradox,Diversity-sensitive grading,Equity-driven educational policies,Interdependence in learning communities,Hierarchical categorization tensions,NaN,Diversity,Educational Policy,Student Evaluation,Educational Principles,Educational Change
2,Article 3,Exploring the use of artificial intelligence (...,Providing effective feedback in large settings...,AI-driven tutoring systems,Competency-based assessment,Pedagogical best practices oversight,Automated feedback variability,Instructor-supervised AI integration,NaN,Technology Integration,Faculty Development,Student Evaluation,Educational Improvement,Computer Uses in Education
3,Article 4,Research impacts and trends in feedback litera...,Given its prominence in maximising feedback pr...,Bibliometric feedback mapping,Authoritative scholarly networks,Disciplinary ecological perspectives,Peer and teacher feedback loops,Linguistic diversity in writing feedback,NaN,Student Evaluation,Faculty Development,Program Evaluation,Educational Change,College Programs
4,Article 5,Can you mitigate gender bias in student evalua...,Student evaluations of teaching can be unrelia...,Gendered evaluative disparities,Implicit bias awareness prompts,Stress/fatigue and feedback timing,Qualitative comment positivity/specificity,Instructor/course characteristics influence,NaN,Student Evaluation of Teacher Performance,Teacher Characteristics,Diversity (Faculty),Educational Policy,Faculty Evaluation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Article 136,Exploring How Culture Matters in Building Resp...,While the vertical transfer process and cultur...,Cultural responsiveness in STEM mentoring,Equity-driven student aspiration support,Humanizing minoritized student experiences,Intersectional identity relevance,Support beyond classroom teaching,NaN,Culturally Relevant Education,Teacher Attitudes,College Programs,Educational Improvement,Policy Formation
136,Article 137,Seed Grant Programs to Promote Community Trans...,"Used in higher education for many decades, see...",Community-oriented funding mechanisms,Inclusive institutional capacity building,Graduate student and staff opportunities,Grassroots innovation and community impact,Structural changes in HE cultures,NaN,Program Implementation,Community Involvement,Educational Improvement,College Programs,Policy Formation
137,Article 138,Supporting Mathematics Instructors’ Transition...,With evidence mounting on the benefits of equi...,Community of practice in PD,Incremental equity-driven change,Interdisciplinary teacher support,"Active, inclusive math pedagogy",Confidence building for faculty,NaN,Inservice Teacher Education,Teacher Collaboration,Educational Improvement,Policy Formation,Faculty Development
138,Article 139,The Role of Language and Literacy Skills in Sc...,Despite the acknowledged impact of early scien...,Literacy-science skill interdependence,"Gender, race/ethnicity and class gaps",Longitudinal STEM achievement patterns,Language-based equity interventions,Integrating literacy in STEM policy,NaN,Culturally Relevant Education,Language Enrollment,Curriculum Development,Educational Improvement,Policy Formation


# Keyword generation

Right now done outside the process (to use O1) but can be integrated into this code.

In [ ]:
Prompt = """
Given this abstract and its title, list 5 domain-specific keywords that best represent its main themes.
Select terms that relate specifically to social aspects of education policy, avoiding overly generic words like ‘education’ or ‘school.
"""

In [ ]:
###You might also prompt for reasons why each keyword was chosen, ensuring greater attention to contextual relevance.

### Limiting to domain specific vocabulary

In [ ]:
eric_terms = [
    "Academic Probation",
    "Academic Rank (Professional)",
    "Academic Records",
    "Academic Standards",
    "Administration",
    "Administrative Change",
    "Administrative Organization",
    "Administrative Policy",
    "Administrative Principles",
    "Administrator Attitudes",
    "Administrator Behavior",
    "Administrator Characteristics",
    "Administrator Effectiveness",
    "Administrator Evaluation",
    "Administrator Qualifications",
    "Administrator Responsibility",
    "Administrator Role",
    "Admission (School)",
    "Admission Criteria",
    "Advanced Placement",
    "Advanced Placement Programs",
    "Advisory Committees",
    "Age Grade Placement",
    "Ancillary School Services",
    "Attendance",
    "Attendance Patterns",
    "Average Daily Attendance",
    "Beginning Teacher Induction",
    "Bus Transportation",
    "Career Pathways",
    "Case Records",
    "Clinical Supervision (of Teachers)",
    "College Administration",
    "College Admission",
    "College Enrollment",
    "College Environment",
    "College Governing Councils",
    "College Instruction",
    "College Planning",
    "College Programs",
    "Common Core State Standards",
    "Comprehensive Programs",
    "Computer Oriented Programs",
    "Confidential Records",
    "Course Selection (Students)",
    "Crisis Management",
    "Critical Path Method",
    "Curriculum Design",
    "Curriculum Development",
    "Curriculum Evaluation",
    "Curriculum Implementation",
    "Degree Requirements",
    "Dress Codes",
    "Dropout Prevention",
    "Dropout Programs",
    "Early Admission",
    "Educational Administration",
    "Educational Environment",
    "Educational Programs (1966 1980)",
    "Educational Specifications (1967 1980)",
    "Emergency Programs",
    "Enrollment",
    "Enrollment Influences",
    "Enrollment Management",
    "Experimental Programs",
    "Faculty College Relationship",
    "Faculty Development",
    "Faculty Evaluation",
    "Faculty Organizations",
    "Faculty Promotion",
    "Faculty Recruitment",
    "Faculty Workload",
    "Flexible Schedules (1967 1980)",
    "Flexible Scheduling",
    "Full Time Equivalency",
    "Grade Inflation",
    "Grade Repetition",
    "Graduation",
    "Graduation Requirements",
    "Guided Pathways",
    "Improvement Programs",
    "Inservice Teacher Education",
    "Institutional Administration",
    "Institutional Characteristics",
    "Institutional Environment",
    "Institutional Evaluation",
    "Institutional Mission",
    "Instructional Leadership",
    "Language Enrollment",
    "Majors (Students)",
    "Management by Objectives",
    "Management Development",
    "Management Systems",
    "Merit Rating",
    "Nongraded Student Evaluation",
    "Noninstructional Responsibility",
    "Organization Size (Groups)",
    "Organizational Change",
    "Organizational Climate",
    "Organizational Communication",
    "Organizational Development",
    "Organizational Effectiveness",
    "Organizational Objectives",
    "Participative Decision Making",
    "Pass Fail Grading",
    "Peer Evaluation",
    "Preschool Evaluation",
    "Program Administration",
    "Program Attitudes",
    "Program Content",
    "Program Design",
    "Program Development",
    "Program Effectiveness",
    "Program Evaluation",
    "Program Implementation",
    "Program Improvement",
    "Program Termination",
    "Publish or Perish Issue",
    "Quality Circles",
    "Recordkeeping",
    "Related Services (Special Education)",
    "Risk Management",
    "Scheduling",
    "School Activities",
    "School Administration",
    "School Based Management",
    "School Catalogs",
    "School Construction",
    "School Councils",
    "School Effectiveness",
    "School Expansion",
    "School Holding Power",
    "School Maintenance",
    "School Organization",
    "School Orientation",
    "School Policy",
    "School Registration",
    "School Safety",
    "School Security",
    "School Shops",
    "School Size",
    "School Space",
    "School Supervision",
    "School Turnaround",
    "School Uniforms",
    "School Violence",
    "Search Committees (Personnel)",
    "Selective Admission",
    "Social Promotion",
    "Special Programs",
    "Staff Development",
    "Staff Meetings",
    "Staff Orientation",
    "Staff Role",
    "Staff Utilization",
    "Student Adjustment",
    "Student Attitudes",
    "Student Attrition",
    "Student Characteristics",
    "Student College Relationship",
    "Student Evaluation",
    "Student Evaluation of Teacher Performance",
    "Student Government",
    "Student Needs",
    "Student Organizations",
    "Student Placement",
    "Student Promotion",
    "Student Records",
    "Student Recruitment",
    "Student Role",
    "Student School Relationship",
    "Student Teacher Attitudes",
    "Student Teacher Evaluation",
    "Student Teaching",
    "Student Transportation",
    "Systems Approach",
    "Systems Concepts (1966 1980)",
    "Systems Development",
    "Teacher Administrator Relationship",
    "Teacher Attendance",
    "Teacher Attitudes",
    "Teacher Background",
    "Teacher Burnout",
    "Teacher Characteristics",
    "Teacher Collaboration",
    "Teacher Discipline",
    "Teacher Dismissal",
    "Teacher Evaluation",
    "Teacher Leadership",
    "Teacher Morale",
    "Teacher Orientation",
    "Teacher Participation",
    "Teacher Promotion",
    "Teacher Qualifications",
    "Teacher Recruitment",
    "Teacher Responsibility",
    "Teacher Role",
    "Teacher Selection",
    "Teacher Student Ratio",
    "Teacher Supervision",
    "Teaching Assignment (1966 1980)",
    "Teaching Load",
    "Teaching Programs (1966 1980)",
    "Textbook Evaluation",
    "Textbook Selection",
    "Total Quality Management",
    "Transfers (1966 1980)",
    "Validated Programs",
    "Academic Freedom",
    "Access to Computers",
    "Access to Education",
    "Accountability",
    "Accreditation (Institutions)",
    "Achievement Gap",
    "African American Achievement",
    "African American Education",
    "Alternative Teacher Certification",
    "American Indian Education",
    "Articulation (Education)",
    "Back to Basics",
    "Bilingual Education",
    "Board Administrator Relationship",
    "Board of Education Policy",
    "Board of Education Role",
    "Boards of Education",
    "Caregiver Training",
    "Certification",
    "Change Agents",
    "Change Strategies",
    "Coeducation",
    "College Attendance",
    "College Choice",
    "College Outcomes Assessment",
    "College Preparation",
    "College Role",
    "College School Cooperation",
    "Community Benefits",
    "Community Control",
    "Community Involvement",
    "Community Support",
    "Compensatory Education",
    "Competency Based Education",
    "Competitive Selection",
    "Compulsory Education",
    "Computer Attitudes",
    "Computer Literacy",
    "Computer Uses in Education",
    "Consortia",
    "Continuation Education (1968 1980)",
    "Contract Training",
    "Controversial Issues (Course Content)",
    "Cooperative Programs",
    "Corporate Education",
    "Correctional Education",
    "Culturally Relevant Education",
    "Declining Enrollment",
    "Demonstration Programs",
    "Discipline",
    "Discipline Policy",
    "Distance Education",
    "Diversity",
    "Diversity (Faculty)",
    "Diversity (Institutional)",
    "Dropout Attitudes",
    "Dropout Characteristics",
    "Dropout Rate",
    "Dual Enrollment",
    "Education",
    "Education Work Relationship",
    "Educational Attitudes",
    "Educational Benefits",
    "Educational Change",
    "Educational Cooperation",
    "Educational Demand",
    "Educational Development",
    "Educational History",
    "Educational Improvement",
    "Educational Innovation",
    "Educational Legislation",
    "Educational Malpractice",
    "Educational Mobility",
    "Educational Needs",
    "Educational Objectives",
    "Educational Opportunities",
    "Educational Philosophy",
    "Educational Planning",
    "Educational Policy",
    "Educational Practices",
    "Educational Principles",
    "Educational Problems (1966 1980)",
    "Educational Quality",
    "Educational Resources",
    "Educational Responsibility",
    "Educational Supply",
    "Educational Technology",
    "Educational Trends",
    "Enrollment Rate",
    "Enrollment Trends",
    "Excellence in Education",
    "Exchange Programs",
    "Expulsion",
    "Faculty Mobility",
    "Family School Relationship",
    "Gifted Education",
    "Governance",
    "Governing Boards",
    "Government School Relationship",
    "Graduation Rate",
    "Hidden Curriculum",
    "Home Programs",
    "Home Schooling",
    "Home Visits",
    "Honor Societies",
    "Individualized Education Programs",
    "Individualized Transition Plans",
    "Informal Education",
    "Information Literacy",
    "Institutional Autonomy",
    "Institutional Cooperation",
    "Institutional Role",
    "Institutional Survival",
    "Integrated Services",
    "Intellectual Freedom",
    "Intercollegiate Cooperation",
    "Intercultural Communication",
    "Intergroup Education",
    "International Educational Exchange",
    "Interschool Communication",
    "Long Range Planning",
    "Mexican American Education",
    "Migrant Adult Education",
    "Migrant Education",
    "Minimum Competencies",
    "Mobile Educational Services",
    "Noncategorical Education",
    "Nonformal Education",
    "Nonschool Educational Programs",
    "Nontraditional Education",
    "Open Educational Resources",
    "Open Enrollment",
    "Outcome Based Education",
    "Outcomes of Education",
    "Outreach Programs",
    "Parent Associations",
    "Parent Caregiver Relationship",
    "Parent Conferences",
    "Parent Education",
    "Parent Grievances",
    "Parent Participation",
    "Parent School Relationship",
    "Parent Student Relationship",
    "Parent Teacher Conferences",
    "Parent Teacher Cooperation",
    "Parent Workshops",
    "Parents as Teachers",
    "Participant Characteristics",
    "Partnerships in Education",
    "Performance Technology",
    "Police School Relationship",
    "Policy",
    "Policy Formation",
    "Politics of Education",
    "Popular Education",
    "Portfolios (Background Materials)",
    "Positive Behavior Supports",
    "Professional Autonomy",
    "Professional Training",
    "Program Validation",
    "Proposal Writing",
    "Public Relations",
    "Public Service",
    "Regional Cooperation",
    "Regional Planning",
    "Regular and Special Education Relationship",
    "Relevance (Education)",
    "Report Cards",
    "Research Training",
    "Residence Requirements",
    "Role of Education",
    "Rural Education",
    "Scholarship",
    "School Attendance Legislation",
    "School Attitudes",
    "School Business Relationship",
    "School Choice",
    "School Closing",
    "School Community Programs",
    "School Community Relationship",
    "School Demography",
    "School District Autonomy",
    "School District Reorganization",
    "School District Size",
    "School Entrance Age",
    "School Involvement",
    "School Law",
    "School Location",
    "School Planning (1966 1980)",
    "School Prayer",
    "School Responsibility",
    "School Restructuring",
    "School Role",
    "School Visitation",
    "Service Learning",
    "Shared Resources and Services",
    "State Boards of Education",
    "State Departments of Education",
    "State School District Relationship",
    "Statewide Planning",
    "Strategic Planning",
    "Student Certification",
    "Student Distribution (1966 1980)",
    "Student Diversity",
    "Student Educational Objectives",
    "Student Empowerment",
    "Student Exchange Programs",
    "Student Leadership",
    "Student Mobility",
    "Student Problems",
    "Student Responsibility",
    "Student Rights",
    "Student Subcultures",
    "Suspension",
    "Teacher Associations",
    "Teacher Certification",
    "Teacher Distribution",
    "Teacher Empowerment",
    "Teacher Exchange Programs",
    "Teacher Militancy",
    "Teacher Persistence",
    "Teacher Placement",
    "Teacher Rights",
    "Teacher Transfer",
    "Teacher Welfare",
    "Technology Integration",
    "Technology Planning",
    "Test Score Decline",
    "Textbook Standards",
    "Track System (Education)",
    "Tracking (1968 1980)",
    "Training",
    "Transfer Policy",
    "Transfer Programs",
    "Transfer Rates (College)",
    "Truancy",
    "Urban Education",
    "Urban Teaching",
    "Veterans Education",
    "Withdrawal (Education)",
    "Womens Education",
    "Zero Tolerance Policy"
]

In [ ]:
prompt_eric = """
Please read the following abstract, titles and identify 5 relevant ERIC thesaurus terms that best represent
its main policy-related themes. Each chosen keyword must be selected exclusively from the predefined ERIC thesaurus terms
list provided below. If no exact match appears, select the closest thematically relevant terms from that list.
If multiple terms are similar, choose the ones that align most clearly with the abstract’s focus on social and educational policy.

"""

# Embedding

In [ ]:
normal_keyword_cols = ["Keyword 1", "Keyword 2", "Keyword 3", "Keyword 4", "Keyword 5"]
eric_keyword_cols = ["Keyword Eric 1", "Keyword Eric 2", "Keyword Eric 3", "Keyword Eric 4", "Keyword Eric 5"]

#Extract list of keywords
normal_keywords_list = df[normal_keyword_cols].values.tolist()
eric_keywords_list = df[eric_keyword_cols].values.tolist()

In [ ]:
model = SentenceTransformer('allenai/scibert_scivocab_uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [ ]:
def embed_keywords(keyword_set):
    filtered = [kw for kw in keyword_set if isinstance(kw, str) and kw.strip() != ""]
    # Embed each keyword separately and then average the embeddings
    if len(filtered) > 0:
        embeddings = model.encode(filtered)
        avg_embedding = np.mean(embeddings, axis=0)
        return avg_embedding
    else:
        # If no valid keywords, return a zero vector or handle accordingly
        return np.zeros(model.get_sentence_embedding_dimension())

In [ ]:
# Compute embeddings for normal keywords
normal_embeddings = []
for row in normal_keywords_list:
    normal_embeddings.append(embed_keywords(row))
normal_embeddings = np.vstack(normal_embeddings)

In [ ]:
# Compute embeddings for ERIC keywords
eric_embeddings = []
for row in eric_keywords_list:
    eric_embeddings.append(embed_keywords(row))
eric_embeddings = np.vstack(eric_embeddings)

# Dimensionality reduction

In [ ]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import umap.umap_ as umap
from sklearn.decomposition import PCA

In [ ]:
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
reduced_normal = reducer.fit_transform(normal_embeddings)
reduced_eric = reducer.fit_transform(eric_embeddings)

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


# Clustering

In [ ]:
!pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 30.3 MB/s eta 0:00:00


In [ ]:
import hdbscan

In [ ]:
# Perform HDBSCAN clustering on normal embeddings
clusterer_normal = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2, metric='euclidean')
normal_labels = clusterer_normal.fit_predict(reduced_normal)

In [ ]:
# Perform HDBSCAN clustering on ERIC embeddings
clusterer_eric = hdbscan.HDBSCAN(min_cluster_size=7, min_samples=2, metric='euclidean')
eric_labels = clusterer_eric.fit_predict(reduced_eric)

In [ ]:
# Add the cluster assignments to the DataFrame
df['Cluster_Normal'] = normal_labels
df['Cluster_ERIC'] = eric_labels

# Download data

In [ ]:
df.to_csv('clusters.csv', index=False)